**Imports**

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from PIL import Image
from shutil import copyfile
from keras.models import Model
from keras.models import load_model

copyfile(src = "/kaggle/input/input-data/augmentor.py", dst = "/kaggle/working/augmentor.py")

from augmentor import *

**Config**

In [2]:
test_csv = '/kaggle/input/UBC-OCEAN/test.csv'
test_images = '/kaggle/input/UBC-OCEAN/test_images/'
model_file = '/kaggle/input/input-data/model.h5'

classes = ['CC', 'EC', 'HGSC', 'LGSC', 'MC']

n_crops = 15

**Functions**

In [3]:
def one_hot_to_values(vector, classes, threshold=0.4):
    values = []
    for one_hot in vector:
        if np.max(one_hot) < threshold:
            value = 'Other'
        else:
            value = classes[np.argmax(one_hot)]
        values.append(value)
    return np.array(values)

def load_crops(idx, it):
    image = Image.open(test_images+str(idx)+'.png')
    crops = []
    for crop, confidence in augmenting_generator(image, n_crops, it=it):
        crops.append(np.array(crop))
        crop.close()
    image.close()
    return np.array(crops).reshape(-1, aug_side_px, aug_side_px, 3)

**Loading model**

In [4]:
model = load_model(model_file)

**Loading test images and making predictions**

In [5]:
test_df = pd.read_csv(test_csv)
n = len(test_df)
predicted_labels = []

def ensemble(predictions, mode='hard_voting'):
    predicted = one_hot_to_values(predictions, classes)
    values, counts = np.unique(predicted, return_counts=True)
    return values[counts.argmax()]

for i, (idx, height) in enumerate(zip(test_df['image_id'], test_df['image_height'])):
    it = 'tma' if height < 5000 else 'wsi'
    crops = load_crops(idx, it)
    predicted_labels.append(ensemble(model.predict(crops, verbose=0)))
    print(f'Images: {i+1} / {n}',end='\r')

test_df["label"] = predicted_labels
submission_df = test_df[["image_id", "label"]]
submission_df.to_csv("submission.csv", index=False)